In [1]:
from math import log

from Phase_1.src.Utils.SimplePositionalIndex import SimplePositionalIndex


class DevelopedPositionalIndex(SimplePositionalIndex):
    def __init__(self, configurations):
        super().__init__(configurations)
        self.total_number_of_documents = len(self.Documents)
        self.document_term_tfidf_dictionary = {}
        self.build_updated_positional_index()
        if configurations.get_config('champions_list'):
            self.champions_list = self.build_champions_list()

    def build_updated_positional_index(self):
        for WORD in self.positional_index_structure.keys():
            for DOC_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                DICTIONARY['tf idf'] = self.get_tf_value(WORD, DOC_URL) * self.get_idf_value(WORD)
                if DOC_URL not in self.document_term_tfidf_dictionary.keys():
                    self.document_term_tfidf_dictionary[DOC_URL] = {WORD: DICTIONARY['tf idf']}
                else:
                    self.document_term_tfidf_dictionary[DOC_URL][WORD] = DICTIONARY['tf idf']

    def get_tf_value(self, word, url):
        return log(1 + self.positional_index_structure[word]['indexes'][url]['number of occurrences in document'])

    def get_idf_value(self, word):
        return log(self.total_number_of_documents / len(self.positional_index_structure[word]['indexes']))

    def build_champions_list(self):
        champions_list = {}
        for WORD in self.positional_index_structure.keys():
            url_tf_dictionary = {}
            for DOC_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                url_tf_dictionary[DOC_URL] = self.get_tf_value(WORD, DOC_URL)
            champions_list_size = int(
                self.config.get_config('champions_lists_ratio') * len(self.positional_index_structure[WORD]['indexes']))
            champions_list[WORD] = sorted(url_tf_dictionary, key=lambda item: item[1], reverse=True)[
                                   :champions_list_size]
        return champions_list


In [2]:
from Phase_1.src.Utils.StopWord import Document
from Phase_1.src.Utils.utilities import read_file, print_results
from Phase_2.src.Utils.DevelopedPositionalIndex import DevelopedPositionalIndex
from Phase_2.src.config import Config

config = Config()
config.set_config('documents_path', '/Volumes/Farshid_SSD/Projects/University/information retrieval/Phase_1/assets/IR_data_news_12k.json')
docs_url, docs_title, docs_content = read_file(config.get_config('documents_path'))

config.set_config('documents',
                  [Document(content, url, title) for url, title, content in zip(docs_url, docs_title, docs_content)])
config.set_config('champions_list', True)

pos_index = DevelopedPositionalIndex(config)

In [11]:
import pickle

def save_index(configurations):
    with open(f'../{configurations.get_config("dump_path")}/updated_pos_index.pkl', 'w') as file_to_write:
        pickle.dump(pos_index, file_to_write, pickle.HIGHEST_PROTOCOL)
        print(f'saved index with name updated_pos_index.pkl')

def load_index(configurations):
    index = None
    with open(f'../{configurations.get_config("dump_path")}/updated_pos_index.pkl', 'rb') as file_to_read:
        index = pickle.load(file_to_read)
    return index

# save_index(config)
# pos_index = load_index(config)

EOFError: Ran out of input

In [ ]:
from collections import Counter
from math import log

from Phase_1.src.Utils.utilities import preprocess_pipeline


class QueryHandler:
    def __init__(self, positional_index, config):
        self.positional_index = positional_index
        self.config = config

    def answer_query(self, query):
        query = preprocess_pipeline(query)
        terms = query.split()
        if not self.config.get_config('champions_list'):
            vector_values = self.tf_idf_calculate_normal(terms)
        else:
            vector_values = self.tf_idf_calculate_champions(terms)
        scores = self.calculate_scores(vector_values)
        results_url = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)[
                           :self.config.get_config('documents_to_show')])
        total_results = self.generate_total_results(results_url)
        return total_results

    def generate_total_results(self, results_url):
        total_results = []
        for URL, SCORE in results_url.items():
            result = self.positional_index.url_to_information[URL]
            result['score'] = SCORE
            result['url'] = URL
            total_results.append(result)
        return total_results

    def tf_idf_calculate_normal(self, terms):
        vector_values = {}
        tf_values = Counter(terms)
        for term in terms:
            positional_index_structure = self.positional_index.positional_index_structure
            if term in positional_index_structure.keys():
                for DOC_URL in positional_index_structure[term]['indexes'].keys():
                    if DOC_URL not in vector_values.keys():
                        vector_values[DOC_URL] = {}
                    vector_values[DOC_URL][term] = \
                        (1 + log(tf_values[term])) * self.positional_index.get_idf_value(term)
        return vector_values

    def tf_idf_calculate_champions(self, terms):
        vector_values = {}
        tf_values = Counter(terms)
        for term in terms:
            if term in self.positional_index.positional_index_structure.keys():
                for DOC_URL in self.positional_index.champions_list[term]:
                    if DOC_URL not in vector_values.keys():
                        vector_values[DOC_URL] = {}
                    vector_values[DOC_URL][term] = \
                        (1 + log(tf_values[term])) * self.positional_index.get_idf_value(term)
        return vector_values

    @staticmethod
    def cosine_similarity(v1, v2):
        dot_product = 0
        for term in v1.keys():
            if term in v2.keys():
                dot_product += v1[term] * v2[term]
        magnitude_v1 = 0
        for term in v1.keys():
            magnitude_v1 += v1[term] ** 2
        magnitude_v2 = 0
        for term in v2.keys():
            magnitude_v2 += v2[term] ** 2
        return dot_product / (magnitude_v1 ** 0.5) / (magnitude_v2 ** 0.5)

    def calculate_scores(self, vector_values):
        scores = {}
        for DOC_URL, TERM_SCORES in vector_values.items():
            scores[DOC_URL] = self.cosine_similarity(vector_values[DOC_URL],
                                                     self.positional_index.document_term_tfidf_dictionary[DOC_URL])
        return scores


query_handler = QueryHandler(pos_index, config)
print_results(query_handler.answer_query('ایران'))

In [ ]:
print_results(query_handler.answer_query('قهرمانی تیم ملی ایران'))

In [ ]:
print_results(query_handler.answer_query('نیشابور'))

In [ ]:
print_results(query_handler.answer_query('هری کین'))